In [1]:
import numpy as np
import zmq
from random import choice

In [2]:
# connect

context = zmq.Context()
socket = context.socket(zmq.DEALER)
identity = 'worker-%d' % (choice([0,1,2,3,4,5,6,7,8,9]))
socket.setsockopt_string(zmq.IDENTITY, identity)
socket.connect("tcp://localhost:5555")
identity

'worker-6'

In [3]:
# create tensors

# 0 tensors.Add (l0_tensor);
socket.send_json({"functionCall":"createTensor", "data": [0,0,1,0,1,1,1,0,1,1,1,1], "shape": [3,3,3,3]})
socket.recv_string()

# 1 tensors.Add (l1_tensor);
socket.send_json({"functionCall":"createTensor", "data": [0,0,0,0], "shape": [4]})
socket.recv_string()

# 2 tensors.Add (syn0_tensor);
np.random.seed(1)
syn0 = 2*np.random.random((3)) - 1
socket.send_json({"functionCall":"createTensor", "data": syn0.tolist(), "shape": [3]})
socket.recv_string()

# 3 tensors.Add (l1_error_tensor);
socket.send_json({"functionCall":"createTensor", "data": [0,0,1,1], "shape": [4]})
socket.recv_string()

# 4 tensors.Add (save_error_tensor);
socket.send_json({"functionCall":"createTensor", "data": [0,0,1,1], "shape": [4]})
socket.recv_string()

'createTensor: OK'

In [4]:
# init

# l1_tensor.init_sigmoid_matrix_multiply (l0_tensor);
tensorIndex = 1
tensorIndexParams = [0]
d = {"functionCall":"init_sigmoid_matrix_multiply","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
socket.send_json(d)
socket.recv_string()

# syn0_tensor.init_add_matrix_multiply (l0_tensor);
tensorIndex = 2
tensorIndexParams = [0]
d = {"functionCall":"init_add_matrix_multiply","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
socket.send_json(d)
socket.recv_string()

# l1_error_tensor.init_weights (save_error_tensor);
tensorIndex = 3
tensorIndexParams = [4]
d = {"functionCall":"init_weights","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
socket.send_json(d)
socket.recv_string()

'init_weights: OK'

In [5]:
# train

for iter in range(100):
    
    # l1_tensor.sigmoid_matrix_multiply (l0_tensor, syn0_tensor);
    tensorIndex = 1
    tensorIndexParams = [0, 2]
    d = {"functionCall":"sigmoid_matrix_multiply","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
    socket.send_json(d)
    socket.recv_string()
    
    # l1_error_tensor.reset_weights ();
    tensorIndex = 3
    d = {"functionCall":"reset_weights","objectType":"tensor","objectIndex":tensorIndex}
    socket.send_json(d)
    socket.recv_string()
    
    # l1_error_tensor.inline_elementwise_subtract (l1_tensor);
    tensorIndex = 3
    tensorIndexParams = [1]
    d = {"functionCall":"inline_elementwise_subtract","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
    socket.send_json(d)
    socket.recv_string()
    
    # l1_error_tensor.multiply_derivative (l1_tensor);
    tensorIndex = 3
    tensorIndexParams = [1]
    d = {"functionCall":"multiply_derivative","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
    socket.send_json(d)
    socket.recv_string()
    
    # syn0_tensor.add_matrix_multiply (l0_tensor, l1_error_tensor);
    tensorIndex = 2
    tensorIndexParams = [0, 3]
    d = {"functionCall":"add_matrix_multiply","objectType":"tensor","objectIndex":tensorIndex,"tensorIndexParams":tensorIndexParams}
    socket.send_json(d)
    socket.recv_string()

In [6]:
# results

print ("Output After Training:")
socket.send_json({"functionCall":"print","objectType":"tensor","objectIndex":1})
print (socket.recv_string())
print ("Weights After Training:")
socket.send_json({"functionCall":"print","objectType":"tensor","objectIndex":2})
print (socket.recv_string())

Output After Training:
0.1103504, 0.09193923, 0.9256488, 0.9104123
Weights After Training:
4.621287, -0.2033847, -2.093253
